In [25]:
from helper_functions import *
import pandas as pd
from pathlib import Path
import time 
import csv
# qiskit_nature imports
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
# imort NUmpy algorithm to solve minumun energy exactly
from qiskit_algorithms import NumPyEigensolver
import numpy as np
from constants import chemistry_molecules

In [26]:
all_molecule_names = list(chemistry_molecules.keys())

In [27]:
path_to_this_file = str(Path.cwd())

In [31]:
# parse the command line arguments
filename                             = 'mapper_precisions'
VQE                                  = ['N']
molecules                            = ['H2','LiH','H2O','NH3']
mappers                              = ['jordan_wigner','parity','bravyi_kitaev','neven']
ansatzes                             = ['EfficientSU2']
measurement_schemes                  = ['pauli_scheme']
Z2Symmetries_list                    = [False]
ansatz_reps                          = [2]


In [29]:
# Create a pandas DataFrame to store the Hamiltonians
data = pd.DataFrame(columns=['molecule',         'z2Symmetries', 'mapping', 'ansatz',
                                'vqe_time',           'hamiltonian',
                                'avg_pauli_weight',  'num_pauli_strings',
                                'num_qubits',        'vqe_energies',
                                'iterations',        'parameters',
                                'error',             'exact_energies',
                                'exact_solution',    'avg_hardware_pauli_weight',
                                'max_pauli_weight',  'max_hrdwr_pauli_weight',
                                'num_parameters',    'gates',
                                'depth',             'ansatz_reps',
                                'classical_time',
                                'accuracies_shots',
                                'vqe_acc'])

In [30]:
for molecule in ['C2H4']:
    for map in ['jordan_wigner']:
        hamiltonian,_,_ = prepare_hamiltonian(molecule_name=molecule, z2symmetry_reduction=True, mapping=map,basis='sto-3g')
        print(molecule,'num_qubits:'+str(hamiltonian.num_qubits))



C2H4 num_qubits:19


In [ ]:
# flush the vqe_results.csv file adn write the header
with open(path_to_this_file+'/../results/'+filename+'.csv','w') as file:
        writer = csv.writer(file)
        writer.writerow(data.columns)
        file.close()


# Iterate over all different parameters and store the results in the DataFrame named data
for molecule in molecules:
    #for reps in ansatz_reps:
    for reps in ansatz_reps:
        for z2sym in Z2Symmetries_list:
            for map in mappers:
                for ansatz in ansatzes:
                    for measurement in measurement_schemes:
                        print("Preparing: molecule:", molecule, "| z2Symmetries:", z2sym, "| mapping:", map, "| ansatz:", ansatz, "| measurement:", measurement)


                        # create the problem hamiltoniansource ~/demo/VQE/bin/activate

                        hamiltonian,num_spatial_orbitals,num_particles = prepare_hamiltonian(molecule_name=molecule, z2symmetry_reduction=z2sym, mapping=map)

                        # retrieve and calculate some useful information
                        num_qubits = hamiltonian.num_qubits
                        num_pauli_strings = len(hamiltonian.paulis)


                        # Pauli weight information
                        pauli_weights = [pauli_weight(pauli) for pauli in hamiltonian.paulis]
                        max_pauli_weight = np.max([pauli_weight(pauli) for pauli in hamiltonian.paulis])
                        avg_pauli_weight = np.mean([pauli_weight(pauli) for pauli in hamiltonian.paulis])                       
                        hardware_pauli_weights = [hardware_pauli_weight(pauli) for pauli in hamiltonian.paulis]
                        max_hardware_pauli_weight = np.max([hardware_pauli_weight(pauli) for pauli in hamiltonian.paulis])


                        # Create the circuit
                        ansatz_circuit = build_ansatz(ansatz_name=ansatz,mapper=map,num_qubits=num_qubits,reps= reps,num_particles=0,num_spatial_orbitals=0)
            
                        # solve exact solution for the mapping
                        #start  = time.time()
                        #exact_solution = np.real(NumPyEigensolver(k=1).compute_eigenvalues(hamiltonian).eigenvalues[0])
                        #print(exact_solution)
                        #class_time_cost = time.time() - start


                        print('Preparation done!')

                            #if molecule=='NH3' or molecule=='C2H4':
                            #    shot = 10_000#,2000]#,5000,10_000,20_000,30_000,40_000,60_000,80_000,100_000]
                            #    vars = variance(hamiltonian,shot,ansatz_circuit)
                            #    shots = [1000,2000,5000,10_000,20_000,40_000,60_000,80_000,100_000,150_000,200_000]
                            #    accuracies = [approx_accracy(hamiltonian,shot,vars) for shot in shots]
                            #else:

                        # Run with 100 shots takes ~13 minutes on a laptop
                        shots = [100] #,2000,5000,10_000,20_000,30_000,40_000,60_000,80_000,100_000]
                        accuracies = [calc_accuracy(hamiltonian,shot,ansatz_circuit,grouping=False) for shot in shots]
                        accuracies_shots = (accuracies,shots)
            

            
                        # store the results in the DataFrame

                        new_row = { 
                                        'molecule': molecule,
                                        'z2Symmetries': str(z2sym),
                                        'mapping': map,
                                        'ansatz': ansatz,
                                        'vqe_time': None,
                                        'hamiltonian': hamiltonian,
                                        'avg_pauli_weight': avg_pauli_weight,
                                        'num_pauli_strings': num_pauli_strings,
                                        'num_qubits': num_qubits,
                                        'vqe_energies': None,
                                        'iterations': None,
                                        'parameters': None,
                                        'error': None,
                                        'exact_energies': None,
                                        'exact_solution': None,
                                        'avg_hardware_pauli_weight': None,
                                        'max_pauli_weight': max_pauli_weight,
                                        'max_hrdwr_pauli_weight': max_hardware_pauli_weight,
                                        'num_parameters': None,
                                        'gates': None,  # assuming cx_gates corresponds to 'gates' in your original DataFrame
                                        'depth': None,
                                        'ansatz_reps': reps,
                                        'classical_time': None,
                                        'accuracies_shots': accuracies_shots,
                                        'vqe_acc': None}


                        # Write to csv
                        with open(path_to_this_file+'/../results/'+filename+'.csv','a') as file:
                            writer = csv.writer(file)
                            writer.writerow(new_row.values())

                        print('One system done and save. Moving to next system...\n')
    print('Moving to the next molecule...\n')

print('Results saved successfully')

Preparing: molecule: H2 | z2Symmetries: False | mapping: jordan_wigner | ansatz: EfficientSU2 | measurement: pauli_scheme
Preparation done!
One system done and save. Moving to next system...

Preparing: molecule: H2 | z2Symmetries: False | mapping: parity | ansatz: EfficientSU2 | measurement: pauli_scheme
Preparation done!
One system done and save. Moving to next system...

Preparing: molecule: H2 | z2Symmetries: False | mapping: bravyi_kitaev | ansatz: EfficientSU2 | measurement: pauli_scheme
Preparation done!
One system done and save. Moving to next system...

Preparing: molecule: H2 | z2Symmetries: False | mapping: neven | ansatz: EfficientSU2 | measurement: pauli_scheme
I hope you are not using kUPCCGSD with Neven mapper
Preparation done!
One system done and save. Moving to next system...

Moving to the next molecule...

Preparing: molecule: LiH | z2Symmetries: False | mapping: jordan_wigner | ansatz: EfficientSU2 | measurement: pauli_scheme
Preparation done!
One system done and sa